In [1]:
!pip install dataflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 970.7 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prep

In [2]:
!python natural_gas_flow.py

python: can't open file '/home/student/Documents/DL_220962089/Lab2/natural_gas_flow.py': [Errno 2] No such file or directory


In [6]:
!pip install scikit-learn



  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 46.8 MB/s eta 0:00:0000:0100:01


In [16]:
import pandas as pd
import torch 
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader

df = pd.read_csv("/home/student/Documents/DL_220962089/Lab2/flat-ui__data-Fri Mar 28 2025.csv")
df =df.dropna()

y = df['Price'].values
x = np.arange(1,len(y),1)

min = y.min()
max = y.max()

y = (y-min)/(max-min)

sequence_length = 10

print(f"Data Length: {len(y)} | Max: {max} | Min: {min} | Seq_length: {sequence_length} -- NORMALIZED")



Data Length: 7088 | Max: 23.86 | Min: 1.05 | Seq_length: 10 -- NORMALIZED


In [15]:
X = []
Y = []
for i in range(0,5099):
    list1=[]
    for j in range(i,i+sequence_length):
        list1.append(y[j])
    X.append(list1)
    Y.append(y[j+1])
X = np.array(X)
Y = np.array(Y)

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(X,Y,test_size=0.10,random_state=42,shuffle=False,stratify=None)

class TimeSeriesData(Dataset):
    def __init__(self, X,Y):
        self.x = torch.tensor(X,dtype=torch.float64)
        self.y = torch.tensor(Y,dtype = torch.float64)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
dataset = TimeSeriesData(x_train,y_train)
train_loader = DataLoader(dataset,batch_size=256,shuffle=True)


class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=1,hidden_size=5,num_layers=1,batch_first=True)
        self.fc1 = nn.Linear(5,1)
    def forward(self,x):
        output,_status = self.rnn(x)
        output = output[:,-1,:]
        output = self.fc1(torch.relu(output))
        return output

model = RNNModel()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
epochs = 1500

for i in range(epochs):
    for j,data in enumerate(train_loader):
        x = data[:][0].to(torch.float32)
        y = data[:][1].to(torch.float32)
        y_pred = model(x.view(-1,sequence_length,1)).reshape(-1)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
    if i%50 == 0:
        print(f"Epoch {i} Loss : ",loss)
    

Epoch 0 Loss :  tensor(0.0781, grad_fn=<MseLossBackward0>)
Epoch 50 Loss :  tensor(0.0078, grad_fn=<MseLossBackward0>)
Epoch 100 Loss :  tensor(0.0078, grad_fn=<MseLossBackward0>)
Epoch 150 Loss :  tensor(0.0081, grad_fn=<MseLossBackward0>)
Epoch 200 Loss :  tensor(0.0058, grad_fn=<MseLossBackward0>)
Epoch 250 Loss :  tensor(0.0005, grad_fn=<MseLossBackward0>)
Epoch 300 Loss :  tensor(0.0103, grad_fn=<MseLossBackward0>)
Epoch 350 Loss :  tensor(0.0089, grad_fn=<MseLossBackward0>)
Epoch 400 Loss :  tensor(0.0030, grad_fn=<MseLossBackward0>)
Epoch 450 Loss :  tensor(0.0010, grad_fn=<MseLossBackward0>)
Epoch 500 Loss :  tensor(0.0066, grad_fn=<MseLossBackward0>)
Epoch 550 Loss :  tensor(0.0017, grad_fn=<MseLossBackward0>)
Epoch 600 Loss :  tensor(0.0010, grad_fn=<MseLossBackward0>)
Epoch 650 Loss :  tensor(0.0012, grad_fn=<MseLossBackward0>)
Epoch 700 Loss :  tensor(0.0033, grad_fn=<MseLossBackward0>)
Epoch 750 Loss :  tensor(0.0004, grad_fn=<MseLossBackward0>)
Epoch 800 Loss :  tensor(0.

In [ ]:
test_set = TimeSeriesData(x_test,y_test)
test_pred = model(test_set[:][0].view(-1,10,1)).view(-1)
plt.plot(test_pred.detach().numpy(),label='predicted')
plt.plot(test_set[:][1].view(-1),label='original')
plt.legend()
plt.show()
#Undo normalization
y = y * (max - min) + min
y_pred = test_pred.detach().numpy() * (max - min) + min
plt.plot(y)
plt.plot(range(len(y)-len(y_pred), len(y)), y_pred)
plt.show()

# Q2